In [62]:
%run "00 - Automatic evaluation - DEV.py"

In [63]:
df_all.sort_values(by='bleu', ascending=False)

,bleu,meteor,ter,elapsed_time,dp_scorer_n,sa_scorer_n,tems_lm_n,referrer_lm_n,txs_lm_n,max_dp,max_sa,max_refs,max_tems
1692738707194431632,60.70,0.443029,0.349523,49.185656,2,2,4,3,4,5,5,2,1
-6392362581655930715,60.68,0.440230,0.350888,52.396428,2,2,4,4,3,5,10,1,2
-8281780742127218386,60.68,0.440030,0.351327,49.959461,2,3,4,4,3,5,10,1,2
8427712258203685535,60.67,0.439775,0.350059,25.549334,2,3,4,4,3,5,5,1,2
-6120210850613917316,60.65,0.440085,0.349864,26.118550,2,2,4,4,3,5,5,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
-2889655248528946001,59.14,0.434655,0.360102,10.136653,2,2,3,4,3,5,10,1,1
-4512488599880313982,58.97,0.433687,0.357128,46.924267,2,2,3,3,3,5,5,2,1
5504290471941121795,58.90,0.432692,0.357713,46.711365,2,2,3,4,3,5,5,2,1
6256836840311939185,58.80,0.433292,0.357470,44.008057,2,3,3,3,3,5,5,2,1


In [64]:
df_all.groupby(['max_dp', 'max_sa', 'max_tems', 'max_refs']).bleu.describe().sort_values('75%', ascending=False)

count       mean       std    min      25%  \
max_dp max_sa max_tems max_refs                                               
5      5      2        1          16.0  60.235625  0.248031  59.84  60.0725   
       10     2        1          16.0  60.248125  0.240506  59.84  60.1100   
       5      1        1          15.0  59.887333  0.529292  59.21  59.3350   
       10     1        1          16.0  59.775625  0.477716  59.14  59.3725   
       5      2        2           3.0  60.076667  0.115902  59.97  60.0150   
              1        2          16.0  59.685000  0.628777  58.73  59.2175   

                                    50%      75%    max  
max_dp max_sa max_tems max_refs                          
5      5      2        1         60.175  60.4175  60.67  
       10     2        1         60.230  60.3675  60.68  
       5      1        1         60.230  60.3600  60.46  
       10     1        1         59.610  60.2325  60.40  
       5      2        2         60.060  60.1300  60.20  
              1        2         59.745  60.0375  60.70

In [65]:
df_all.groupby(['dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'txs_lm_n']).bleu.describe().sort_values('75%', ascending=False)

count       mean       std    min  \
dp_scorer_n sa_scorer_n tems_lm_n txs_lm_n                                      
2           2           4         3          11.0  60.283636  0.350550  59.57   
                                  4          10.0  60.328000  0.196344  60.08   
            3           4         3           9.0  60.184444  0.465272  59.30   
                                  4          10.0  60.221000  0.261213  59.59   
                        3         3          10.0  59.528000  0.632575  58.73   
            2           3         3          11.0  59.622727  0.577288  58.90   
                                  4          11.0  59.891818  0.227192  59.48   
            3           3         4          10.0  59.744000  0.309595  59.32   

                                                25%     50%      75%    max  
dp_scorer_n sa_scorer_n tems_lm_n txs_lm_n                                   
2           2           4         3         60.2100  60.330  60.5250  60.68  
                                  4         60.1475  60.375  60.4175  60.70  
            3           4         3         60.1800  60.240  60.3900  60.68  
                                  4         60.1550  60.290  60.3250  60.60  
                        3         3         59.1725  59.235  60.1775  60.42  
            2           3         3         59.1800  59.350  60.1200  60.50  
                                  4         59.7700  59.890  60.0550  60.24  
            3           3         4         59.4950  59.770  59.9150  60.23